In [1]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

Using custom data configuration default
Reusing dataset medical_questions_pairs (/home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae)


In [2]:
from transformers import AutoTokenizer, BertForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

model = BertForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [3]:
dataset = dataset["train"].train_test_split(test_size=0.2)

Loading cached split indices for dataset at /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-ddbba2ce54e35142.arrow and /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-4f60af92002c21dd.arrow


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 2438
    })
    test: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 610
    })
})

In [5]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [6]:
def tokenize(batch):
    return tokenizer(batch['question_1']+"[SEP]"+batch["question_2"], padding="max_length", truncation=True, max_length=512)

In [7]:
train_dataset = train_dataset.map(tokenize, batched=False)
test_dataset = test_dataset.map(tokenize, batched=False)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Loading cached processed dataset at /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-fef6ae0e3568df64.arrow
Loading cached processed dataset at /home/parker/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-33cbe069815d15c1.arrow


In [8]:
len(train_dataset[0]["input_ids"])

512

In [9]:
len(train_dataset[1]["input_ids"])

512

In [10]:
train_dataset

Dataset({
    features: ['attention_mask', 'dr_id', 'input_ids', 'label', 'question_1', 'question_2', 'token_type_ids'],
    num_rows: 2438
})

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [12]:
trainer.train()

In [ ]:
trainer.evaluate()